# Using Broadcast Variables and Accumulators

This small tutorial enhances the WordCount exmaple by stop words, i.e. words that should be ignored. We will apply two techniques in this example:
1. Broadcast Variables. As the name already suggests, data will be transported to all workers in the cluster.
2. Accumulators. Those are handy counter variables which can be used to derive some processing metrics

In [2]:
# First define a list of uninteresting words, the stop words
stopwords = frozenset(['a','the','an','it','is','are'])

# Broadcast this list to all worker processes in the cluster
stopwords_bc = sc.broadcast(stopwords)

# Create two accumulators for counting processed words
stopword_count = sc.accumulator(0)
regular_count = sc.accumulator(0)

# Define a filter function
def filter_word(w):
    # Check if a given word is in the list of stopwords
    if w in stopwords_bc.value:
        stopword_count.add(1)
        return False
    else:
        regular_count.add(1)
        return True

In [3]:
text = sc.textFile('/user/cloudera/alice.txt')
words = text.flatMap(lambda x: x.split()) \
    .filter(filter_word) \
    .map(lambda x: (x,1)) \
    .reduceByKey(lambda x,y: x+y) \
    .sortBy(lambda x: x[1], ascending=False) \
    .map(lambda (k,v): k + ':' + str(v))
words.saveAsTextFile('/user/cloudera/alice_counts3')

In [4]:
# Print processing metrics
print stopword_count.value
print regular_count.value

2881
26580
